In [11]:
# IMPORT
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [12]:
# DATA
data = pd.read_csv('train.csv')
print("Đã tải dữ liệu train.csv thành công!")
    
print("\n--- Tên tất cả các trường ---")
print(data.columns.tolist())

Đã tải dữ liệu train.csv thành công!

--- Tên tất cả các trường ---
['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF

TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
# 1. Nhóm Vị trí & Đặc điểm lô đất 

In [ ]:
# 2. NHÓM QUY MÔ & KHÔNG GIAN SỬ DỤNG
SPACE_FEILDS = [
    'BedroomAbvGr', 'TotRmsAbvGrd', 'FullBath', 'HalfBath', 'BsmtFullBath',
    'BsmtHalfBath', 'GrLivArea', '1stFlrSF', '2ndFlrSF', 'TotalBsmtSF',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'LowQualFinSF',
    'GarageArea', 'GarageCars', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    'ScreenPorch', '3SsnPorch'
]

X_size_space = data[SPACE_FEILDS].copy()

# --- Pipeline 1: Standard Scaling Lần 1 ---
standard1_scaling_space = Pipeline(steps=[
    ('standard_scaler_1', StandardScaler())
])

data_standard1_scaled_space = standard1_scaling_space.fit_transform(X_size_space)
print("Hoàn thành Chuẩn hóa Standard Lần 1.")


# --- Pipeline 2: Standard Scaling Lần 2 ---
standard2_scaling_space = Pipeline(steps=[
    ('standard_scaler_2', StandardScaler())
])

# Áp dụng Pipeline 2 (lên kết quả của Pipeline 1)
data_standard_scaled_space = standard2_scaling_space.fit_transform(data_standard1_scaled_space)
print("Hoàn thành Chuẩn hóa Standard Lần 2.")


# Tạo DataFrame cuối cùng cho kết quả
df_num_space_scaled = pd.DataFrame(
    data_standard_scaled_space,
    columns=[f'{col}' for col in SPACE_FEILDS]
)

print("\n--- Kết quả Tiền xử lý (2 Pipeline Standard Scaling) ---")
print(df_num_space_scaled.head())

Hoàn thành Chuẩn hóa Standard Lần 1.
Hoàn thành Chuẩn hóa Standard Lần 2 (Trên kết quả lần 1).

--- Kết quả Tiền xử lý (2 Pipeline Standard Scaling) ---
   BedroomAbvGr  TotRmsAbvGrd  FullBath  HalfBath  BsmtFullBath  BsmtHalfBath  \
0      0.163779      0.912210  0.789741  1.227585      1.107810     -0.241061   
1      0.163779     -0.318683  0.789741 -0.761621     -0.819964      3.948809   
2      0.163779     -0.318683  0.789741  1.227585      1.107810     -0.241061   
3      0.163779      0.296763 -1.026041 -0.761621      1.107810     -0.241061   
4      1.390023      1.527656  0.789741  1.227585      1.107810     -0.241061   

   GrLivArea  1stFlrSF  2ndFlrSF  TotalBsmtSF  ...  BsmtFinSF2  BsmtUnfSF  \
0   0.370333 -0.793434  1.161852    -0.459303  ...   -0.288653  -0.944591   
1  -0.482512  0.257140 -0.795163     0.466465  ...   -0.288653  -0.641228   
2   0.515013 -0.627826  1.189351    -0.313369  ...   -0.288653  -0.301643   
3   0.383659 -0.521734  0.937276    -0.687324  ...  

In [ ]:
# 3. Nhóm Chất lượng & Tình trạng công trình 

In [ ]:
# 4. Nhóm Tiện ích, điều kiện bán & yếu tố thời gian 